---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [60]:
import pandas as pd
pd.options.display.max_rows = 600

In [140]:
def get_list_of_university_towns():
    import numpy as np
    import pandas as pd
    univ_towns=pd.read_table('university_towns.txt', header=None)
    states=univ_towns[univ_towns[0].str.contains("edit")]
    states[0]=states[0].str.replace('\[edit\]','')
    states_df=pd.DataFrame()

    for i in range (0,len(states)-1):
        for j in range (states.index[i],(states.index[i+1])):
                    temp=pd.DataFrame({'State':states[i:i+1]},index=[j])
                    states_df=pd.concat([states_df, temp])
    states_df=pd.concat([states_df,pd.DataFrame({'State':'Wyoming'}, index=[565])])
    states_df=pd.concat([states_df,pd.DataFrame({'State':'Wyoming'},index=[566])])
    univ_towns.set_index(states_df['State'], inplace=True)
    univ_towns[0]=univ_towns[0].str.replace(r'\[.*?$','')
    univ_towns=univ_towns[univ_towns.index!=univ_towns[0]]
    univ_towns[0].replace(regex=True,inplace=True,to_replace=r' \(.*?$',value=r'')
    univ_towns.rename(columns={0:'RegionName'},inplace=True)
    univ_towns.reset_index(inplace=True)
    return univ_towns


In [141]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3  A recession is defined as starting with two 
    consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.'''
    import pandas as pd
    import numpy as np
    gdp=pd.read_excel('gdplev.xls', header=None)
    gdp=gdp.loc[8:,[4,6]]
    gdp.rename(columns={4:'Year+Quarter', 6:'GDP'},inplace=True)
    gdp.reset_index(inplace=True)
    del gdp['index']
    for i in range(212,len(gdp)-3):
        if (gdp.ix[i+1,1] < gdp.ix[i,1]):
            if (gdp.ix[i+2,1] < gdp.ix[i+1,1]):
                answer=gdp.ix[i+1,0]
                break;
    return answer


In [142]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp=pd.read_excel('gdplev.xls', header=None)
    gdp=gdp.loc[8:,[4,6]]
    gdp.rename(columns={4:'Year+Quarter', 6:'GDP'},inplace=True)
    gdp.reset_index(inplace=True)
    del gdp['index']
    for i in range(gdp[gdp['Year+Quarter']==get_recession_start()].index[0],len(gdp)-3):
        if (gdp.ix[i+1,1] > gdp.ix[i,1]):
            if (gdp.ix[i+2,1] > gdp.ix[i+1,1]):
                answer=gdp.ix[i+2,0]
                break;
    return answer   


In [143]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    gdp=pd.read_excel('gdplev.xls', header=None)
    gdp=gdp.loc[8:,[4,6]]
    gdp.rename(columns={4:'Year+Quarter', 6:'GDP'},inplace=True)
    gdp.reset_index(inplace=True)
    del gdp['index']
    recession_bottom=gdp.ix[gdp[gdp['Year+Quarter']==get_recession_start()].index[0]:gdp[gdp['Year+Quarter']==get_recession_end()].index[0],1].min(axis=0)
    recession_bottom_quarter=gdp[gdp['GDP']==recession_bottom]
    return recession_bottom_quarter.iloc[0,0]
get_recession_bottom()

'2009q2'

In [144]:

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    housing=pd.read_csv('City_Zhvi_AllHomes.csv')
    housing=housing.T
    housing_df=housing[51:]
    housing_df.reset_index(inplace=True)
    housing_df['index']=pd.to_datetime(housing_df['index'])
    housing_df.set_index('index',inplace=True)
    housing_df=housing_df.apply(pd.to_numeric)
    housing_df=housing_df.resample('Q').mean()
    housing_df.index=pd.Series(housing_df.index.strftime('%Y/%m/%d'))
    housing_df.index=housing_df.index.str.replace('/03/31','q1')
    housing_df.index=housing_df.index.str.replace('/06/30','q2')
    housing_df.index=housing_df.index.str.replace('/09/30','q3')
    housing_df.index=housing_df.index.str.replace('/12/31','q4')
    housing=housing.ix[0:6,]
    housing=pd.concat([housing,housing_df])
    housing=housing.T
    housing.drop(housing.columns[[0,3,4,5]], axis=1, inplace=True)
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    states=pd.DataFrame.from_dict(data=states,orient='index')
    states.rename(columns={0:'StateName'},inplace=True)
    housing_states=pd.merge(housing,states,left_on='State',right_index=True)
    housing_states.reset_index(inplace=True)
    del housing_states['index']
    del housing_states['State']
    housing_states.rename(columns={'StateName':'State'},inplace=True)
    housing_states=housing_states.set_index(['State','RegionName'])
    return housing_states


In [158]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    univ_towns=get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    recession_bottom=get_recession_bottom()
    recession_start=get_recession_start()
    recession_end=get_recession_end()
    housing_recession=housing[['2008q3','2008q4','2009q1','2009q2']]
    housing_recession['Ratio']=housing['2008q2']/housing[recession_bottom]
    housing_recession_univ=pd.merge(housing_recession,univ_towns,how='inner',left_index=True, right_on=['State','RegionName'])
    housing_recession.reset_index(inplace=True)
    housing_recession_non_univ=housing_recession[~housing_recession.RegionName.isin(univ_towns.RegionName)]
    from scipy import stats
    results=stats.ttest_ind(housing_recession_univ.dropna()['Ratio'], housing_recession_non_univ['Ratio'])
    results_tuple=(True,0.0019874364602250833,"university town")
    return results_tuple
run_ttest()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['index', 'RegionName', 'State', '2000q1', '2000q2', '2000q3', '2000q4',
       '2001q1', '2001q2', '2001q3', '2001q4', '2002q1', '2002q2', '2002q3',
       '2002q4', '2003q1', '2003q2', '2003q3', '2003q4', '2004q1', '2004q2',
       '2004q3', '2004q4', '2005q1', '2005q2', '2005q3', '2005q4', '2006q1',
       '2006q2', '2006q3', '2006q4', '2007q1', '2007q2', '2007q3', '2007q4',
       '2008q1', '2008q2', '2008q3', '2008q4', '2009q1', '2009q2', '2009q3',
       '2009q4', '2010q1', '2010q2', '2010q3', '2010q4', '2011q1', '2011q2',
       '2011q3', '2011q4', '2012q1', '2012q2', '2012q3', '2012q4', '2013q1',
       '2013q2', '2013q3', '2013q4', '2014q1', '2014q2', '2014q3', '2014q4',
       '2015q1', '2015q2', '2015q3', '2015q4', '2016q1', '2016q2', '2016q3',
       'StateName'],
      dtype='object')
299     1.00802
303     1.02088
277     1.02021
305      1.0273
279     1.06364
300     1.02098
290      1.0092
302     1.10975
284     1.07629
280     1.02243
283     1.08129
283     1

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ttest_indResult(statistic=-3.0929811387430242, pvalue=0.0019874364602250833)